In [1]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from PIL import Image
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/data-yolo.zip'

Mounted at /content/drive


In [3]:
import shutil
import os
import zipfile

# extract zip file
download_folder = "/content"
zip_name = "data-yolo.zip"
os.makedirs(download_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(download_folder)

print(f"Extraction complete! Files saved to {download_folder}")

Extraction complete! Files saved to /content


In [8]:
# split data into training and validation, still in yolo format

import os
import shutil
import random

# Paths
dataset_path = download_folder
image_dir = os.path.join(dataset_path, "images")
label_dir = os.path.join(dataset_path, "labels")

# Output paths
train_img = os.path.join(dataset_path, "images/train")
val_img = os.path.join(dataset_path, "images/val")
train_lbl = os.path.join(dataset_path, "yolo/labels/train")
val_lbl = os.path.join(dataset_path, "yolo/labels/val")

# Create train/val folders
for d in [train_img, val_img, train_lbl, val_lbl]:
    os.makedirs(d, exist_ok=True)

# List images and shuffle
images = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])  # Adjust for other image formats
random.shuffle(images)

# Split 80% train, 20% val
split_idx = int(0.8 * len(images))
train_files, val_files = images[:split_idx], images[split_idx:]

# Move files
for file_list, img_dest, lbl_dest in [(train_files, train_img, train_lbl), (val_files, val_img, val_lbl)]:
    for file in file_list:
        shutil.move(os.path.join(image_dir, file), os.path.join(img_dest, file))
        label_file = file.replace(".jpg", ".txt")  # Assuming YOLO format
        if os.path.exists(os.path.join(label_dir, label_file)):
            shutil.move(os.path.join(label_dir, label_file), os.path.join(lbl_dest, label_file))

print("Dataset successfully split into training and validation sets!")

yolo_train_labels = train_lbl
yolo_val_labels = val_lbl

Dataset successfully split into training and validation sets!


In [11]:
# Converting from YOLO annotations to annotations for Faster RCNN

import os
import json
import glob
from PIL import Image

#classes = ["Exploding Kittens", "Munchkin", "Poker", "Uno"]

rcnn_train_labels = os.path.join(dataset_path, "labels/train")
rcnn_val_labels = os.path.join(dataset_path, "labels/val")

# Create directories if they don't exist
os.makedirs(rcnn_train_labels, exist_ok=True)
os.makedirs(rcnn_val_labels, exist_ok=True)

def yolo_to_rcnn_bbox(x_center, y_center, width, height, img_w, img_h):
    x_center *= img_w
    y_center *= img_h
    width *= img_w
    height *= img_h

    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2

    return [x_min, y_min, x_max, y_max]


def convert_yolo_folder_to_rcnn(yolo_folder,img_folder,out_folder):
  for label_file in os.listdir(yolo_folder):
    if not label_file.endswith('.txt'):
        continue

    img_name = label_file.replace('.txt', '.jpg')
    img_path = os.path.join(img_folder, img_name)
    label_path = os.path.join(yolo_folder, label_file)

    if not os.path.exists(img_path):
        print(f"Image not found for {label_file}, skipping.")
        continue

    img = Image.open(img_path)
    img_w, img_h = img.size

    with open(label_path, 'r') as f:
        lines = f.readlines()

    rcnn_lines = []
    for line in lines:
      class_id, x_c, y_c, w, h = map(float, line.strip().split())
      x_min, y_min, x_max, y_max = yolo_to_rcnn_bbox(x_c, y_c, w, h, img_w, img_h)
      rcnn_lines.append(f"{int(class_id)} {x_min} {y_min} {x_max} {y_max}")

    output_file = os.path.join(out_folder, label_file)
    with open(output_file, 'w') as f:
      f.write("\n".join(rcnn_lines))

    print(f'Converted labels from {yolo_folder} to {out_folder}')
  return out_folder

train_lbl = convert_yolo_folder_to_rcnn(yolo_train_labels, train_img, rcnn_train_labels)
val_lbl = convert_yolo_folder_to_rcnn(yolo_val_labels, val_img, rcnn_val_labels)

Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/labels/train to /content/labels/train
Converted labels from /content/yolo/la

In [12]:
# Read class names from classes.txt
classes_txt_path = os.path.join(dataset_path, "classes.txt")
with open(classes_txt_path, "r") as f:
    class_names = [line.strip() for line in f.readlines()]

# Generate dataset.yaml
dataset_yaml_path = os.path.join(dataset_path, "dataset.yaml")
with open(dataset_yaml_path, "w") as f:
    f.write(f"train: {os.path.join(dataset_path, 'images/train')}\n")
    f.write(f"val: {os.path.join(dataset_path, 'images/val')}\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")

print('\nFile contents:\n')
!cat /content/dataset.yaml


File contents:

train: /content/images/train
val: /content/images/val
nc: 4
names: ['Exploding Kittens', 'Munchkin', 'Poker', 'Uno']


In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

Using device: cuda


NameError: name 'images_dir' is not defined